In [1]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph

maup.progress.enabled = True

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# state = New Hampshire
state_ab = "nh"

## Data
1. Download all the data in directory "nh_data"
2. Extract them all

In [4]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2022/NHCongDists2022.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2022/NHSenateDists2022.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2022/NHHouseDists2022_base.shp".format(data_folder, state_ab)

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    if maup.doctor(df):
        print('smart_repair successful')
            
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('smart_repair failed')
    return df

In [49]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)
    
    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")
    
    # assign the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        district_index = election_df.at[precinct_index, dist_name]
        if district_index in dist_df.index:
            election_df.at[precinct_index, dist_name] = dist_df.at[district_index, col_name]
        else:
            print(f"District index {district_index} not found in dist_df")
    return election_df

In [7]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [8]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [9]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [10]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)

    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated

    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
        
    return df

### Read the census data

In [11]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [12]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [13]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [14]:
maup.doctor(population_df)

100%|██████████| 31948/31948 [00:52<00:00, 604.64it/s] 


True

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['P0020005'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['P0020006'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['P0020007'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['P0020008'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['P0020009'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['P0020010'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['P0020011'], 1)

In [16]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
                    'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
                    'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
                                        'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [17]:
population_df.rename(columns=rename_dict, inplace = True)

In [18]:
cong_df = gpd.read_file(cd_data)
if maup.doctor(cong_df) != True:
    cong_df = do_smart_repair(cong_df)

100%|██████████| 2/2 [00:00<00:00, 56.15it/s]


## Read the vest 20 data

Now using it as a "base pricinct"

In [19]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

## Check if vest20 can be used as base

In [20]:
vest20 = gpd.read_file(vest20_data)
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|██████████| 321/321 [00:00<00:00, 352.60it/s]


In [21]:
vest20.columns

Index(['STATEFP20', 'COUNTYFP20', 'VTDST20', 'GEOID20', 'NAME20', 'NAMELSAD20',
       'G20PREDBID', 'G20PRERTRU', 'G20PRELJOR', 'G20USSDSHA', 'G20USSRMES',
       'G20USSLODO', 'G20USSOWRI', 'G20GOVDFEL', 'G20GOVRSUN', 'G20GOVLPER',
       'G20GOVOWRI', 'geometry'],
      dtype='object')

In [22]:
start_col = 6
vest_base_data = vest20
year = '20'

In [23]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [24]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|██████████| 321/321 [00:04<00:00, 79.19it/s]


In [25]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [26]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

In [27]:
election_df.columns

Index(['COUNTYFP20', 'GEOID20', 'GOV20D', 'GOV20O', 'GOV20R', 'NAME20',
       'NAMELSAD20', 'PRE20D', 'PRE20O', 'PRE20R', 'STATEFP20', 'USS20D',
       'USS20O', 'USS20R', 'VTDST20', 'geometry', 'TOTPOP', 'HISP', 'NH_WHITE',
       'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE',
       'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER',
       'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP',
       'NHPIVAP', 'OTHERVAP', '2MOREVAP'],
      dtype='object')

## Check if population agrees

In [28]:
pd.DataFrame({
    'pop_col': pop_col,
    'population_df': population_df[pop_col].sum(), 
    'vest_base': vest_base[pop_col].sum(),
    'equal': [x == y for x, y in zip(population_df[pop_col].sum(), vest_base[pop_col].sum())]
})

,pop_col,population_df,vest_base,equal
TOTPOP,TOTPOP,1377529,1377529,True
HISP,HISP,59454,59454,True
NH_WHITE,NH_WHITE,1200649,1200649,True
NH_BLACK,NH_BLACK,18655,18655,True
NH_AMIN,NH_AMIN,2299,2299,True
NH_ASIAN,NH_ASIAN,35604,35604,True
NH_NHPI,NH_NHPI,388,388,True
NH_OTHER,NH_OTHER,5916,5916,True
NH_2MORE,NH_2MORE,54564,54564,True
H_WHITE,H_WHITE,15554,15554,True


## Add more vest data

In [29]:
vest18 = gpd.read_file(vest18_data)
vest16 = gpd.read_file(vest16_data)

In [30]:
vest18.columns

Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAMELSAD', 'NAME', 'G18GOVRSUN',
       'G18GOVDKEL', 'G18GOVIJAR', 'G18GOVOWRI', 'geometry'],
      dtype='object')

In [31]:
vest16.columns

Index(['STATEFP', 'COUNTYFP', 'VTDST', 'NAMELSAD', 'NAME', 'G16PRERTRU',
       'G16PREDCLI', 'G16PRELJOH', 'G16PREGSTE', 'G16PREOFUE', 'G16PREOWRI',
       'G16USSRAYO', 'G16USSDHAS', 'G16USSLCHA', 'G16USSIDAY', 'G16USSOWRI',
       'G16GOVRSUN', 'G16GOVDVAN', 'G16GOVLABR', 'G16GOVOWRI', 'geometry'],
      dtype='object')

In [32]:
# check the result here
election_df = add_vest(vest18, election_df, '18', population_df, 5)

100%|██████████| 321/321 [00:03<00:00, 80.34it/s]


population agrees


In [33]:
election_df.columns

Index(['2MOREVAP', 'AMINVAP', 'ASIANVAP', 'BVAP', 'COUNTYFP20', 'GEOID20',
       'GOV18D', 'GOV18O', 'GOV18R', 'GOV20D', 'GOV20O', 'GOV20R', 'HISP',
       'HVAP', 'H_2MORE', 'H_AMIN', 'H_ASIAN', 'H_BLACK', 'H_NHPI', 'H_OTHER',
       'H_WHITE', 'NAME20', 'NAMELSAD20', 'NHPIVAP', 'NH_2MORE', 'NH_AMIN',
       'NH_ASIAN', 'NH_BLACK', 'NH_NHPI', 'NH_OTHER', 'NH_WHITE', 'OTHERVAP',
       'PRE20D', 'PRE20O', 'PRE20R', 'STATEFP20', 'TOTPOP', 'USS20D', 'USS20O',
       'USS20R', 'VAP', 'VTDST20', 'WVAP', 'geometry'],
      dtype='object')

In [34]:
election_df = add_vest(vest16, election_df, '16', population_df, 5)

100%|██████████| 321/321 [00:04<00:00, 79.65it/s]


population agrees


##  Add the district data

In [35]:
send = gpd.read_file(send_data)
hdist = gpd.read_file(hdist_data)

In [36]:
cong_df.head()

,CONG2022,Shape_Leng,Shape_Area,geometry
0,1,2.239231e+06,6.441009e+10,"POLYGON ((1156228.000 651107.312, 1156342.000 ..."
1,2,3.884610e+06,1.943597e+11,"POLYGON ((1134197.000 1023109.000, 1134513.000..."


In [50]:
cong_df = cong_df.to_crs("EPSG:4269")
election_df = add_district(cong_df, "CD", election_df, "CONG2022")

100%|██████████| 2/2 [00:00<00:00,  5.84it/s]

District index nan not found in dist_df


In [38]:
send.head()

,OBJECTID,Senate2022,Shape_Leng,Shape_Area,geometry
0,1,1,2.030669e+06,6.663473e+10,"POLYGON ((1134197.000 1023109.000, 1134513.000..."
1,2,2,7.738720e+05,1.436829e+10,"POLYGON ((995330.000 554368.125, 995431.313 55..."
2,3,3,1.183870e+06,3.822333e+10,"POLYGON ((1155297.000 671660.313, 1155348.000 ..."
3,4,4,3.094276e+05,2.654994e+09,"POLYGON ((1174379.000 265454.406, 1174527.001 ..."
4,5,5,9.319983e+05,1.791363e+10,"POLYGON ((877198.875 528286.375, 878506.000 52..."


In [41]:
send = send.to_crs("EPSG:4269")
election_df = add_district(send, "SEND", election_df, "Senate2022")

100%|██████████| 24/24 [00:00<00:00, 37.39it/s]


KeyError: nan

In [42]:
hdist.head()

,OBJECTID,Shape_Leng,Shape_Area,BaseHse22,geometry
0,1,271270.424961,1.522614e+09,BE1,"POLYGON ((1042924.000 437050.906, 1043272.000 ..."
1,2,233012.370304,1.521100e+09,BE2,"POLYGON ((1044227.000 435828.000, 1044369.000 ..."
2,3,187244.074755,1.716795e+09,BE3,"POLYGON ((1022843.000 361524.000, 1022940.000 ..."
3,4,133602.928523,8.898246e+08,BE4,"POLYGON ((1031791.000 373463.094, 1031872.000 ..."
4,5,152431.066247,6.842493e+08,BE5,"POLYGON ((1045615.000 391128.094, 1045241.000 ..."


In [43]:
hdist = hdist.to_crs("EPSG:4269")
election_df = add_district(hdist, "HDIST", election_df, "OBJECTID")

100%|██████████| 164/164 [00:00<00:00, 315.66it/s]


There are 129 overlaps.
There are 1219 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████| 2613/2613 [00:01<00:00, 1408.13it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


100%|██████████| 164/164 [00:00<00:00, 321.25it/s]


smart_repair successful


100%|██████████| 164/164 [00:01<00:00, 137.31it/s]


KeyError: nan

In [ ]:
election_df.columns

### Put the base precinct year after the precinct information column

In [ ]:
'STATEFP', 'COUNTYFP', 'COUNTYNAME', 'VTDST', 'NAME',
base_columns = {}
if 'COUNTYFP' + year not in election_df.columns:
    base_columns = {
        'STATEFP':'STATEFP'+year,
        'COUNTYFP':'COUNTYFP'+year,
        'COUNTYNAME':'COUNTYNAME'+year,
        'VTDST':'VTDST'+year,
        'NAME':'NAME'+year}
election_df.rename(columns=base_columns, inplace = True)

In [ ]:
election_df.columns

In [ ]:
# reorder the columns
fixed_columns = [
    'STATEFP'+year,
    'COUNTYFP'+year,
    'COUNTYNAME'+year,
    'VTDST'+year,
    'NAME'+year,
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [ ]:
import os

# store the result in directory "il"
directory = "./{}".format(state_ab)
if not os.path.exists(directory):
    os.makedirs(directory)

shapefile_path = "./{}/{}.shp".format(state_ab, state_ab)
geojson_path = './{}/{}.geojson'.format(state_ab, state_ab)
json_path = "./{}/{}.json".format(state_ab, state_ab)

# Check if the shapefile or geojson file already exists
if os.path.exists(shapefile_path):
    os.remove(shapefile_path)
if os.path.exists(geojson_path):
    os.remove(geojson_path)

election_df.to_file(shapefile_path)
election_df.to_file(geojson_path, driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file(shapefile_path, ignore_errors=True)
graph.to_json(json_path)

In [ ]:
shapefile_path = "./{}/{}.shp".format(state_ab, state_ab)
shape=gpd.read_file(shapefile_path)
shape.plot()